In [105]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from helpers import get_abspath, save_dataset
from matplotlib import cm
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA

winepath = get_abspath('winequality.csv', 'data/experiments')
seismicpath = get_abspath('seismic_bumps.csv', 'data/experiments')
wine = np.loadtxt(winepath, delimiter=',')
seismic = np.loadtxt(seismicpath, delimiter=',')

# split data into X and y
wX = wine[:, :-1]
wY = wine[:, -1]
sX = seismic[:, :-1]
sY = seismic[:, -1]
wDims = wX.shape[1]
sDims = sX.shape[1]
evp = 0.85

r = range(0, wDims)
pca = PCA(random_state=0, svd_solver='full', n_components=wDims)
comps = pca.fit_transform(wX)  # get principal components
ev = pd.Series(pca.explained_variance_, index=r, name='ev')
evr = pd.Series(pca.explained_variance_ratio_, index=r, name='evr')
evrc = evr.rename('evr_cum').cumsum()
res = comps[:, :evrc.where(evrc > evp).idxmin() + 1]
evars = pd.concat((ev, evr, evrc), axis=1)

In [106]:
evars

,ev,evr,evr_cum
0,3.809984,0.317450,0.317450
1,2.529137,0.210729,0.528179
2,1.561740,0.130125,0.658304
3,0.970988,0.080903,0.739207
4,0.723925,0.060318,0.799525
5,0.613635,0.051128,0.850653
6,0.536449,0.044697,0.895350
7,0.503062,0.041915,0.937266
8,0.352507,0.029371,0.966637
9,0.256872,0.021403,0.988039
